In [ ]:
# default_exp pybrms

In [ ]:
#export
#hide
import typing
import pandas as pd
import numpy as np
import cmdstanpy
import re

import rpy2.robjects.packages as rpackages
from rpy2.robjects import default_converter, pandas2ri, numpy2ri, ListVector, DataFrame, StrVector
from rpy2.robjects.conversion import localconverter

try:
    brms = rpackages.importr("brms")
except:
    utils = rpackages.importr("utils")
    utils.chooseCRANmirror(ind=1)
    utils.install_packages(StrVector(('brms',)))
    brms = rpackages.importr("brms")

# Documentation

In [ ]:
#export
def get_brms_data(dataset_name:str):
    "A helper function for importing different datasets included in brms."
    with localconverter(default_converter + pandas2ri.converter + numpy2ri.converter) as cv:
        return pd.DataFrame(rpackages.data(brms).fetch(dataset_name)[dataset_name])

In [ ]:
assert isinstance(get_brms_data("epilepsy"),pd.DataFrame)
assert isinstance(get_brms_data("kidney"),pd.DataFrame)
assert isinstance(get_brms_data("inhaler"),pd.DataFrame)

In [ ]:
#export
def _convert_python_to_R(data: typing.Union[dict, pd.DataFrame]):
    """
    Converts a python object to an R object brms can handle:
    * python dict      ->   R list
    * python dataframe ->   R dataframe
    """
    with localconverter(default_converter + pandas2ri.converter + numpy2ri.converter) as cv:
        if isinstance(data, pd.DataFrame):
            return DataFrame(data)
        elif isinstance(data, dict):
            return ListVector(data)
        else:
            raise ValueError("Data should be either a pandas dataframe or a dictionary")

In [ ]:
assert isinstance(_convert_python_to_R(dict(a=1, b=2)),ListVector)
assert isinstance(_convert_python_to_R(get_brms_data("inhaler")),DataFrame)

In [ ]:
#export
def get_stan_code(
    formula: str,
    data: typing.Union[dict, pd.DataFrame],
    priors: list,
    family: str,
    sample_prior: str="no"
):
    if len(priors)>0:
        return brms.make_stancode(
            formula=formula, data=data, prior=priors, family=family, sample_prior=sample_prior
        )[0]
    else:
        return brms.make_stancode(
            formula=formula, data=data, family=family, sample_prior=sample_prior
        )[0]

In [ ]:
#export
def _convert_R_to_python(
    formula: str, data: typing.Union[dict, pd.DataFrame], family: str
):
    # calls brms to preprocess the data; returns an R ListVector
    model_data = brms.make_standata(formula, data, family=family)
    
    # a context manager for conversion between R objects and python/pandas/numpy
    # we're not activating it globally because it conflicts with creation of priors
    with localconverter(default_converter + pandas2ri.converter + numpy2ri.converter) as cv:
        model_data = dict(model_data.items())
    return model_data

In [ ]:
#export
def _coerce_types(stan_code, stan_data):
    pat_data = re.compile(r'(?<=data {)[^}]*')
    pat_identifiers = re.compile(r'([\w]+)')

    # extract the data block and separate lines
    data_lines = pat_data.findall(stan_code)[0].split('\n')
    
    # remove commets, <>-style bounds and []-style data size declarations 
    data_lines_no_comments = [l.split('//')[0] for l in data_lines]
    data_lines_no_bounds = [re.sub('<[^>]+>', '',l) for l in data_lines_no_comments]
    data_lines_no_sizes = [re.sub('\[[^>]+\]', '',l) for l in data_lines_no_bounds]

    # extract identifiers - first one should be the type, last one should be the name
    identifiers = [pat_identifiers.findall(l) for l in data_lines_no_sizes]
    var_types = [l[0] for l in identifiers if len(l)>0]
    var_names = [l[-1] for l in identifiers if len(l)>0]
    var_dict = dict(zip(var_names, var_types))

    # coerce integers to int and 1-size arrays to scalars
    for k,v in stan_data.items():
        if k in var_names and var_dict[k]=="int":
            stan_data[k] = v.astype(int) 
        if v.size==1:
            stan_data[k] = stan_data[k][0]
    return stan_data
    

In [ ]:
#export
def fit(
    formula: str,
    stan_file: str,
    data: typing.Union[dict, pd.DataFrame],
    priors: typing.Optional[list] = None,
    family: str = "gaussian",
    sample_prior: str = "no",
    sample:bool = "yes",
    model_kwargs: typing.Optional[dict] = None
     **fit_kwargs,
):
    if priors is None:
        priors = []
    if model_kwargs is None:
        model_kwargs = {}

    formula = brms.bf(formula)
    data = _convert_python_to_R(data)

    if len(priors)>0:
        brms_prior = brms.prior_string(*priors[0])
        for p in priors[1:]:
            brms_prior = brms_prior + brms.prior_string(*p)
        assert brms.is_brmsprior(brms_prior)
    else:
        brms_prior = []

    model_code = get_stan_code(
        formula=formula,
        data=data,
        family=family,
        priors=brms_prior,
        sample_prior=sample_prior,
    )
    model_data = _convert_R_to_python(formula, data, family)
    model_data = _coerce_types(model_code, model_data)
    
    with open(stan_file, "w") as f:
        f.write(model_code)

    sm = cmdstanpy.CmdStanModel(stan_file=stan_file, **model_kwargs)
    if sample==False:
        return sm
    else:
        fit = sm.sample(data=model_data, **fit_kwargs)
        return fit

In [ ]:
from nbdev.showdoc import *

In [ ]:
from nbdev.export import *
notebook2script()

Converted core.ipynb.
Converted index.ipynb.
